In [3]:
# LIBRERIAS
import matplotlib
from ipywidgets import interact,widgets
import numpy as np
import matplotlib.pylab as plt
import celluloid as cell  #animations
import pandas as pd
import os
from scipy.interpolate import CubicSpline
import math

#sns.set_theme()
#matplotlib.rc('xtick', labels
# ize=20) 
#matplotlib.rc('ytick', labelsize=20) 
font = {'weight' : 'bold',
        'size'   : 10}

matplotlib.rc('font', **font)

path=os.getcwd()



def fourierT(f):

    return (np.fft.fftshift(np.fft.fft(np.fft.fftshift(f))))


def fourierI(f):

    return (np.fft.fftshift(np.fft.ifft(np.fft.fftshift(f))))



In [4]:
def new_method(I,phi,gamma,S,T,dt,h):
    '''
    Este codigo calcula un paso espacial. Recibe un array
    con la intensidad y la fase. Modifica ese array para dar lugar al
    siguiente paso espacial.

    '''

   
    In_j = I  # Recibe In_j
    phin_j = phi  # Recibe phin_j

    Inew = []
    phinew = []

    # Ciclo para calcular cada componente del array que contendra el siguiente
    # paso espacial
    for j in range(len(I)):

        Itn_j = (In_j[j]+In_j[j-1])*0.5
        deltaIn_j = In_j[j]-In_j[j-1]
        deltatheta = phin_j[j]-phin_j[j-1]
        deltataun_j = min([abs(deltatheta),abs(deltatheta+2*np.pi),abs(deltatheta-2*np.pi)])


        if abs(deltatheta) == deltataun_j:
            deltaphin_j = deltatheta

        elif  abs(deltatheta+2*np.pi) == deltataun_j:
            deltaphin_j = deltatheta + 2*np.pi
        else:  
            deltaphin_j = deltatheta - 2*np.pi


        In1_j = In_j[j]-h/dt*(3*gamma*S*Itn_j*deltaIn_j)
        phitn1_j = phin_j[j]-h/dt*(gamma*T*deltaIn_j
                                +gamma*S*Itn_j*deltaphin_j
                                )
        phin1_j = phitn1_j + h*gamma*In1_j

        Inew.append(In1_j) # Intensidad en el siguiente paso espacial
        phinew.append(phin1_j) #vfase en el siguiente paso espacial

    return np.sqrt(np.array(Inew))*np.exp(1j*np.array(phinew))

def fourier_split_mod(A1,V,S,T,gamma,betas,lenght,dT,steps):

    
    h = lenght/steps
    
    betas_freq = sum((1j)**(i+1)*betas[i]*(1j*V)**(i+2)/math.factorial(i+2) for i in range(0,len(betas)))
    
    UD1 = np.exp(-h*0.5*betas_freq,dtype=complex)

    for i in range(steps):


        A1a = fourierI(UD1*fourierT(A1))

        N1 = new_method(np.abs(A1a)**2,np.angle(A1a),gamma,S,T,dT,h)

        A1aa = N1
     
        A1 = fourierI(UD1*fourierT(A1aa))
       
    return A1
